#Ejercicios de MapReduce

Quispe Yahuira ronaldo

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 37.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=fbb5f394a71fc70c1a37fe53945c0a88bfcecde45d685457135a38884e5e71e2
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc =SparkContext()

In [10]:
import math

#1 BagofWords

Cambia la letras a minusculas,Filtra letra mayores a 2,Quita espacios en blanco

In [11]:
def bagofwords(Doc):
    x=Doc.strip().lower().split()
    return list(filter(lambda x:len(x)>2,x))
Corpus=["La Informática es la rama de la Ingeniería que estudia el hardware",  
        "las redes de datos y el software necesarios para tratar información de forma automática"]
Texto=sc.parallelize(Corpus,6)
Texto=Texto.map(bagofwords)
Texto.take(2)

[['informática', 'rama', 'ingeniería', 'que', 'estudia', 'hardware'],
 ['las',
  'redes',
  'datos',
  'software',
  'necesarios',
  'para',
  'tratar',
  'información',
  'forma',
  'automática']]

#2.TF

In [12]:

def TF(Doc):
    total=len(Doc)#Obtenemos el total de palabras de doc
    DocSR=list(set(Doc))#Lista sin repeticiones
    mapeo=list(map(lambda x:(x,Doc.count(x)),DocSR))#Obtenemos el tf para cada palabra
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/total)),mapeo))
    return (tf)
Corpus=["La Informática es la rama de la Ingeniería que estudia el hardware",  
        "las redes de datos y el software necesarios para tratar información de forma automática"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoTF.take(2)

[[('que', 0.22184874961635637),
  ('ingeniería', 0.22184874961635637),
  ('rama', 0.22184874961635637),
  ('estudia', 0.22184874961635637),
  ('informática', 0.22184874961635637),
  ('hardware', 0.22184874961635637)],
 [('automática', 0.0),
  ('las', 0.0),
  ('redes', 0.0),
  ('tratar', 0.0),
  ('software', 0.0),
  ('datos', 0.0),
  ('para', 0.0),
  ('necesarios', 0.0),
  ('forma', 0.0),
  ('información', 0.0)]]

#3.IDF

In [13]:
def IDF(Doc,corpus):
    DocP=Doc.flatMap(lambda x:x)
    DocP=DocP.map(lambda x:x[0]).distinct()
    total=corpus.count() #Total documentos en corpus
    corpus2=corpus.collect()
    IDF=DocP.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus2)))#Extraer cantida de veces de cada palabra en los documentos
    IDF2=IDF.flatMap(lambda x:x)
    IDF2=IDF2.groupByKey().map(lambda x:(x[0],sum(x[1])))
    IDF2=IDF2.map(lambda x:(x[0],math.log10(1+total/x[1])))
    return IDF2 #corpus con IDF de cada palabra sin repeticiones
    Corpus=["La Informática es la rama de la Ingeniería que estudia el hardware",  
        "las redes de datos y el software necesarios para tratar información de forma automática"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoIDF.collect()

[('informática', 0.47712125471966244),
 ('hardware', 0.47712125471966244),
 ('automática', 0.47712125471966244),
 ('tratar', 0.47712125471966244),
 ('datos', 0.47712125471966244),
 ('necesarios', 0.47712125471966244),
 ('las', 0.47712125471966244),
 ('redes', 0.47712125471966244),
 ('software', 0.47712125471966244),
 ('forma', 0.47712125471966244),
 ('que', 0.47712125471966244),
 ('ingeniería', 0.47712125471966244),
 ('rama', 0.47712125471966244),
 ('para', 0.47712125471966244),
 ('estudia', 0.47712125471966244),
 ('información', 0.47712125471966244)]

4. TF-IDF

In [15]:
def RecIDF(word,corpusIDF):
        L=corpusIDF
        EL=list(filter(lambda x:x[0]==word,L))
        return EL[0][1]
def TF_IDF(TextoTF,TextoIDF):
    corpusIDF=TextoIDF.collect()
    TFIDF=TextoTF.map(lambda x:list(map((lambda y:(y[0],y[1]*RecIDF(y[0],corpusIDF))),x)))
    return  TFIDF #texto con el TFIDF de cada palabra en DOC
Corpus=["La Informática es la rama de la Ingeniería que estudia el hardware",  
        "las redes de datos y el software necesarios para tratar información de forma automática"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bagofwords)
TextoTF=Texto.map(TF)
TextoIDF=IDF(TextoTF,Texto)
TextoTFIDF=TF_IDF(TextoTF,TextoIDF)
TextoTFIDF.collect()

[[('que', 0.10584875377494418),
  ('ingeniería', 0.10584875377494418),
  ('rama', 0.10584875377494418),
  ('estudia', 0.10584875377494418),
  ('informática', 0.10584875377494418),
  ('hardware', 0.10584875377494418)],
 [('automática', 0.0),
  ('las', 0.0),
  ('redes', 0.0),
  ('tratar', 0.0),
  ('software', 0.0),
  ('datos', 0.0),
  ('para', 0.0),
  ('necesarios', 0.0),
  ('forma', 0.0),
  ('información', 0.0)]]